In [1]:
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import UnstructuredFileLoader, DirectoryLoader
import pandas as pd
import re
import re
import uuid
from llama_index.llms.openai import OpenAI
from tqdm.notebook import tqdm
import os

In [ ]:
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"

In [ ]:
loader = DirectoryLoader('../data/', glob="**/*.pdf", show_progress=True, loader_cls=UnstructuredFileLoader)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
databases = {}
for doc in documents:
    source = doc.metadata['source']
    match = re.search(r'\/([A-Za-z_]+)\.pdf', source)
    if match:
        municipality_name = match.group(1)
    docs = text_splitter.split_documents([doc])
    databases[municipality_name] = docs

In [ ]:
def keep_first_item(dictionary):
    if not dictionary:
        return {}
    first_key = next(iter(dictionary))
    return {first_key: dictionary[first_key]}
filtered_dict = keep_first_item(databases)
print(filtered_dict)

In [ ]:
def keep_specific_instances(dictionary, indices):
    result = {}
    for key, value in dictionary.items():
        if isinstance(value, list):
            instances = [value[i] for i in indices if 0 <= i < len(value)]
            result[key] = instances
        else:
            result[key] = value
    return result

indices_to_keep = [10, 11, 12]
filtered_dict = keep_specific_instances(filtered_dict, indices_to_keep)


In [ ]:
def generate_queries(
    corpus,
    num_questions_per_chunk=1,
    prompt_template=None,
    verbose=False,
):
    """
    Automatisk generer hypotetiske spørsmål som kan besvares med dokumentet i korpuset.
    """
    llm = OpenAI(model='gpt-4')

    prompt_template = prompt_template or """\
    Kontekstinformasjonen er nedenfor.

    ---------------------
    {context_str}
    ---------------------

    Gitt kontekstinformasjonen og ingen tidligere kunnskap.
    Generer bare spørsmål basert på forespørselen nedenfor.
    Du er en innbygger som har spørsmål angående planbestemmelser og planreguleringer i kommunen du bor i. Oppgaven din er både å sette opp {num_questions_per_chunk} spørsmål som en typisk innbygger kan lure på fra dokumentene som er gitt, men også svare på dette spørsmålet som en ekspert der svaret ditt inneholder den faktiske informasjonen uten noen henvisning til hvor i dokumentet svaret ligger.
    Spørsmålene bør være varierte i naturen på tvers av dokumentet. Begrens spørsmålene og svarene til den kontekstinformasjonen som er gitt.

    """    
    queries = {}
    relevant_docs = {}
    for municipality_name, text_chunks in tqdm(corpus.items()):
        for chunk in text_chunks:
            query = prompt_template.format(context_str=chunk, num_questions_per_chunk=num_questions_per_chunk)
            response = llm.complete(query)

            result = str(response).strip().split("\n")
            questions = [
                re.sub(r"^\d+[\).\s]", "", question).strip() for question in result
            ]
            questions = [question for question in questions if len(question) > 0]
            for question in questions:
                question_id = str(uuid.uuid4())
                queries[question_id] = question
                relevant_docs[question_id] = [municipality_name]
    return queries, relevant_docs

In [ ]:
train_queries, train_relevant_docs = generate_queries(
    filtered_dict,
    num_questions_per_chunk=1,
    prompt_template=None,
    verbose=False,
)

In [ ]:
data_list = list(train_queries.items())

keys = []
questions = []
answers = []


for i in range(0, len(data_list), 2):
    key = data_list[i][0]
    question = data_list[i][1].split(': ')[1]
    answer = data_list[i+1][1].split(': ')[1]
    keys.append(key)
    questions.append(question)
    answers.append(answer)


df = pd.DataFrame({
    'nøkkel': keys,
    'spørsmål': questions,
    'svar': answers
})

df['kommunenavn'] = [train_relevant_docs[key][0] for key in df['nøkkel']]

In [ ]:
df_without_key = df.drop(columns=['nøkkel'])
df_without_key.to_csv('Synthetic_data_path', index=False)